In [1]:
%cd ../

/u02/thanhhm/research-idea


In [2]:
import time

from plasma.parallel_processing import communicators, queues
from plasma.parallel_processing.communicators.tree_flow import TreeFlow
from plasma.functional import partials, AutoPipe

/u02/thanhhm/research-idea/.env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def compute(x):
    time.sleep(2)
    return [x * 2] * 2

def compute2(x):
    time.sleep(2)
    return x - 1

class ComputeClass(AutoPipe):

    def __init__(self, hihi, haha):
        super().__init__()
    
    def run(self, *inputs, **kwargs):
        return queues.Signal.IGNORE

In [4]:
class Flow(TreeFlow):

    def __init__(self):
        super().__init__()

        self.compute = communicators.processor.Yield(compute)
        self.compute2 = compute2
        self.compute3 = ComputeClass(1, 2)
        self.printer1 = partials(print, 'step 1')
        self.printer2 = partials(print, 'step 2')

        self.register_chains(
            (queues.ProcessQueue(5), None, 'compute'),
            (queues.ProcessQueue(5), 'compute', 'compute2'),
            (queues.TransferQueue(), 'compute', 'printer1'),
            (queues.TransferQueue(), 'compute2', 'printer2'),
            (queues.ProcessQueue(), 'compute', 'compute3'),
            (queues.ThreadQueue(), 'compute3', None),
        )

flow = Flow()
flow

[ProcessQueue(runner=1)]

  |-(compute:<function compute at 0x7fabaf9b99e0>)

    |-[ProcessQueue(runner=1)]

      |-(compute2:<function compute2 at 0x7fabaf80fba0>)

        |-[TransferQueue(runner=1)]

          |-(printer2:print(step 2,*,**))

    |-[TransferQueue(runner=1)]

      |-(printer1:print(step 1,*,**))

    |-[ProcessQueue(runner=1)]

      |-(compute3:ComputeClass)

        |-[ThreadQueue(runner=1)]

In [5]:
with flow:
   for q in flow.inputs.values():
      for i in range(10):
         q.put(i)

step 1 4
step 1 2
step 1 6
step 1 4
step 1 2
step 1 0
step 1 6
step 1 8
step 1 0
step 1 8
step 1step 2 3
 12
step 1 12
step 1 10
step 1 10
step 1 14
step 1 16
step 1 16
step 1 14
step 1 18
step 1 18
step 2 1
step 2 5
step 2 -1
step 2 1
step 2 5
step 2 3
step 2 -1
step 2 7
step 2 7
step 2 9
step 2 11
step 2 11
step 2 9
step 2 15
step 2 15
step 2 13
step 2 13
step 2 17
step 2 17


In [6]:
flow.inputs

{'compute': ProcessQueue(
   persistent=False,
   n=5,
 )}

In [7]:
flow.outputs

{'compute3': ThreadQueue(
   persistent=False,
   n=1,
 )}